In [1]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import re
import ipywidgets as widgets
from IPython.display import display

In [2]:
nodes_df = pd.read_csv('network/nodes.csv').rename(columns={'# index': 'index', ' _pos': 'pos'})
edges_df = pd.read_csv('network/edges.csv').rename(columns={'# source': 'source', ' target': 'target', ' weight': 'weight', ' layer': 'layer'})
props_df = pd.read_csv('network/gprops.csv', escapechar = '\\').rename(columns={'# prop_name': 'prop_name', ' value': 'value'})

In [3]:
edges_df.weight.unique()

array([1])

In [4]:
edges_df.drop(columns=['weight'], inplace=True)

In [5]:
props_df

,prop_name,value
0,name,7th_graders
1,description,A small multiplex network of friendships among...
2,citation,"['M. Vickers and S. Chan, ""Representing Classr..."
3,url,https://manliodedomenico.com/data.php
4,tags,"['Social', 'Offline', 'Multilayer', 'Unweighte..."
5,layer_key,"{1: 'get_on_with', 2: 'best_friends', 3: 'work..."


In [6]:
edges_grouped = edges_df.groupby(['source', 'target']).agg(lambda x: ', '.join(map(str, x))).reset_index()

pairs = set(zip(edges_grouped['source'], edges_grouped['target'], edges_grouped['layer']))

edges_grouped['mutual'] = edges_grouped.apply(
    lambda row: 1 if (row['target'], row['source'], row['layer']) in pairs else 0, axis=1
)

edges_count = edges_grouped.groupby(['layer', 'mutual']).size()

In [13]:
layer_key = {'3': 'Chciałby współpracować z', 
             '1, 3': 'Dogaduje się, Chciałby współpracować z',
             '1, 2': 'Najlepszy przyjaciel, Dogaduje się z',
             '1, 2, 3': 'Najlepszy przyjaciel, Dogaduje się, Chciałby współpracować z',
             '1': 'Dogaduje się z'}

def draw_graph(selected_layer):
    
    G = nx.DiGraph()
    
    for _, row in nodes_df.iterrows():
        pattern = re.compile(r'array\(\[([^\]]+)\]\)')
        pos = pattern.sub(lambda m: f"({m.group(1).strip()})", row['pos'])
        G.add_node(row['index'] + 1, pos=eval(pos))
    
    filtered_edges = edges_grouped[edges_grouped['layer'] == selected_layer]
    
    for _, row in filtered_edges.iterrows():
        G.add_edge(row['source'] + 1, row['target'] + 1, layer=row['layer'])

    pos = nx.get_node_attributes(G, 'pos')

    fig, ax = plt.subplots(figsize=(14, 12), facecolor="white")
    
    node_colors = ['lightblue' if node <= 12 else 'pink' for node in nodes_df['index']]
    edge_colors = ['green' if row['mutual'] == 1 else 'gray' for _, row in filtered_edges.iterrows()]
    
    nx.draw(
        G,
        pos,
        with_labels=True,
        width=1.5,
        node_size=1500,
        node_color=node_colors,
        edge_color = edge_colors,
        font_size=10,
        arrows=True,
        arrowsize=20,
        ax=ax
    )
    
    node_handles = [
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='lightblue', markersize=10, label='Chłopiec'),
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='pink', markersize=10, label='Dziewczynka'),
    ]
    
    edge_handles = [
        plt.Line2D([0], [0], color='green', lw=4, label=f"Wzajemna relacja ({edges_count.get((selected_layer, 1), 0)})"),
        plt.Line2D([0], [0], color='gray', lw=4, label=f"Relacja jednostronna ({edges_count[selected_layer, 0]})"),
    ]
    
    plt.legend(handles=node_handles + edge_handles, loc='best')
    
    plt.text(0.5, -0.05, "Uczniowie nominowali kolegów z klasy do trzech różnych aktywności (kogo lubisz w klasie, kto jest twoim najlepszym przyjacielem, i z kim chciałbyś współpracować). Graf pokazuje odpowiedzi uczniów, z strzałkami wskazującymi na wierzchołek, który został nominowany przez wierzchołek źródłowy oraz informację ile zostało udzielonych odpowiedzi danego typu, z podziałem na to czy relacja jest wzajemna czy jednostronna.", 
             ha="center", va="center", fontsize=10, color="black", transform=ax.transAxes, wrap = True)
    
    plt.title(f"Sieć przyjaźni dla odpowiedzi: {layer_key[selected_layer]}", fontsize=15)
        
    file_path = f"plots/layer_{selected_layer}.png"
    plt.savefig(file_path, format="PNG", bbox_inches="tight", )
      
    plt.show()

layer_selector = widgets.Dropdown(
    options=[(layer_key[key], key) for key in layer_key],
    description='Odpowiedź:',
    value = '1, 2, 3'
)

widgets.interactive(draw_graph, selected_layer=layer_selector)

interactive(children=(Dropdown(description='Odpowiedź:', index=3, options=(('Chciałby współpracować z', '3'), …